<a href="https://colab.research.google.com/github/Mithilesh0079/Data-Structures-and-Algorithms-Algorithmic-Toolbox-week2/blob/master/Read_coco_data_and_save.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import cv2 
import os
print("Numpy Version is ", np.__version__)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
annFile='gdrive/My Drive/COCO/instances_train2017.json'
coco=COCO(annFile)

In [0]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
cat_temp= nms
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [0]:
#bear 1, elephant 2, giraffe 3, zebra 4, other 5
other_cat=['bird','cat','dog','horse','sheep','cow','kite','wine glass','cup','banana','apple','chair','laptop','mouse','remote', 'keyboard', 'cell phone',]

In [0]:
cat1to4= {'elephant', 'bear', 'giraffe', 'zebra'}

In [0]:
animal= []
target= []
bbox=   []
for cat in cat1to4:
  catIds = coco.getCatIds(catNms=cat);
  imgIds = coco.getImgIds(catIds=catIds );
  for i in range(0,min(len(imgIds),1100)):
      img = coco.loadImgs(imgIds[i])[0]
      I = io.imread(img['coco_url'])
      I1= cv2.resize(I,(224,224),interpolation = cv2.INTER_NEAREST)
      annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
      anns = coco.loadAnns(annIds)
      temp=[]
      for i in range(len(anns)):
        x_top_to_w= anns[i]['bbox'][0]/anns[i]['bbox'][2]
        y_top_to_h= anns[i]['bbox'][1]/anns[i]['bbox'][3]
        w_new= anns[i]['bbox'][2]*224/I.shape[1]
        h_new= anns[i]['bbox'][3]*224/I.shape[0]
        x_new= x_top_to_w*w_new
        y_new= y_top_to_h*h_new
        box_temp= [x_new, y_new, w_new, h_new]
        temp.append(box_temp)
      bbox.append(temp)
      animal.append(I1)
      target.append(cat)

In [0]:
import random
n= random.randint(0,10)
print(bbox[n])
plt.imshow(animal[n])

In [0]:
for i in range(1100):
  n= random.randint(0,len(other_cat)-1)
  cat= other_cat[n]
  catIds = coco.getCatIds(catNms=cat);
  imgIds = coco.getImgIds(catIds=catIds );
  p= random.randint(0,min(len(imgIds),500))
  img = coco.loadImgs(imgIds[p])[0]
  I = io.imread(img['coco_url'])
  I1= cv2.resize(I,(224,224),interpolation = cv2.INTER_NEAREST)
  annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
  anns = coco.loadAnns(annIds)
  temp=[[0.0,0.0,0.0,0.0]]
  bbox.append(temp)
  animal.append(I1)
  target.append('other')

In [0]:
print(len(animal))
print(len(bbox))
print(len(target))

In [0]:
X=[]
Y= []
bbox1=[]
for i in range(len(target)):
  x= animal[i].shape
  s=0
  for j in x:
    s= s+j;
  if(s==451): #removing 2d images
    X.append(animal[i])
    Y.append(target[i])
    bbox1.append(bbox[i])

In [0]:
print(len(X))
print(len(Y))
print(len(bbox1))

In [0]:
imdata =np.array(X)
bbox= np.array(bbox1)
target= Y

In [0]:
y= np.zeros([len(target),5],dtype= int)

In [0]:
for i in range(len(target)):
  if target[i]=='elephant':
    y[i][1]= 1
  elif target[i]== 'bear':
    y[i][0]=1
  elif target[i]=='giraffe':
    y[i][2]= 1
  elif target[i]=='zebra':
    y[i][3]=1
  else:
    y[i][4]= 1

In [0]:
np.save('gdrive/My Drive/COCO/X_data_1100perclass.npy', imdata)
np.save('gdrive/My Drive/COCO/Y_data_1100perclass.npy',y)
np.save('gdrive/My Drive/COCO/bbox_1100perclass.npy', bbox)